# H5 generation - random sampling , normalization

## Revision
#### 20181031
* resampling 방식 수정 --> Scale 과 offset 이 물체의 크기비율을 반영할 수 있도록 수정
```python

x_min = float(data[:,0].min())
x_max = float(data[:,0].max())
y_min = float(data[:,1].min())
y_max = float(data[:,1].max())
z_min = float(data[:,2].min())
z_max = float(data[:,2].max())

scale_val = np.max([ x_max - x_min , y_max - y_min , z_max - z_min ])
offset_val = np.min([x_min, y_min, z_min])

data[:,0] = data[:,0] - min_val
data[:,1] = data[:,1] - min_val
data[:,2] = data[:,2] - min_val

data[:,0] = data[:,0] / float(scale_val)
data[:,1] = data[:,1] / float(scale_val)
data[:,2] = data[:,2] / float(scale_val)
```

#### 20181102
* Re-arrange for running by one click 

## Dataset
#### 20181102
* [dataset_20181102_001](https://drive.google.com/open?id=18EmELMDij5CTaOdiWWUv2qxAYUkj1oQI)
 - db_aug_v4_20181031_ndb_aug_v2_20181020_rs2
 - db_aug_v4_v_100_20181031_ndb_aug_v2_20181020_rs2
 - db_aug_v4_v_200_20181031_ndb_aug_v2_20181020_rs2
 - db_aug_v4_v_300_20181031_ndb_aug_v2_20181020_rs2
 - db_aug_v4_v_400_20181031_ndb_aug_v2_20181020_rs2
 - db_aug_v4_v_500_20181031_ndb_aug_v2_20181020_rs2


In [19]:
""" path """
load_data_path = '../dataset/db_aug_v4_v_500_20181031.pickle'
load_ndata_path = '../dataset/ndb_aug_v2_20181020.pickle'

In [20]:
import numpy as np

import pickle

import os, sys

import h5py

import os, sys

from tqdm import tqdm, trange

sys.path.append( os.path.abspath('../scripts') )
from utils import *

num_sample = 256
num_random_sample = 2

load_data_path = os.path.abspath(load_data_path)
load_ndata_path = os.path.abspath(load_ndata_path)

""" load data and resampling """

object_list = []
object_list.append('world_RangeRoverSportSVRSimple')
object_list.append('world_BmwX5Simple')
object_list.append('world_CitroenCZeroSimple')
object_list.append('world_LincolnMKZSimple')
object_list.append('world_ToyotaPriusSimple')
object_list.append('world_BusSimple')
object_list.append('world_TruckSimple')
object_list.append('world_Pedestrian')
object_list.append('world_MotorbikeSimple')
object_list.append('world_ScooterSimple')

object_label_list = [1,1,1,1,1, 2,2, 3, 4,4]

"""
unkown: 0
cars: 1
trucks: 2
pedestrians: 3
bikes: 4
""" 

## Load positive data
# load_data_path = os.path.abspath('../dataset/db_aug_v2_20181019.pickle')
# load_data_path = os.path.abspath('../dataset/db_aug_v2_v_500_20181019.pickle')
# load_data_path = os.path.abspath('../dataset/db_aug_v4_20181031.pickle')

with open(load_data_path, 'rb') as f:
    db_aug_test = pickle.load(f)   
    
## Load negative data

with open(load_ndata_path, 'rb') as f:
    ndb = pickle.load(f)   
    
""" data / label parsing from db """

## Positive data
data = []
label = []

for i in range(len(object_list)):
    
    key = object_list[i]
    cls = object_label_list[i]
    
    print key, cls
    sys.stdout.flush()
    
    i = 0
    for cloud in tqdm(db_aug_test[key]):
        
        for _ in range(num_random_sample): 
            
            data.append(NormalizeResample(cloud, num_sample))
            label.append(cls)
            
sys.stdout.flush()
print "Only positive:"
print "data:", len(data)
print "label:", len(label)
sys.stdout.flush()

## Negative data
for i in trange(  len( ndb['unknown'] )  ):
    
    cloud = ndb['unknown'][i]
    cls = 0
        
    for _ in range(num_random_sample): 

        data.append(NormalizeResample(cloud, num_sample))
        label.append(cls)

sys.stdout.flush()
print "With negative:"
print "data:", len(data)
print "label:", len(label)
sys.stdout.flush()

out_path = load_data_path.split('.')[0] + "_" + load_ndata_path.split('/')[-1].split('.')[0] + "_rs2"
if not os.path.exists(out_path):
    os.mkdir(out_path)

## save h5 file    
print "load_data_path:", load_data_path
print "load_ndata_path:", load_ndata_path
print "out_path:", out_path
save_h5_files(data, label, out_path, data_dtype, label_dtype)

world_RangeRoverSportSVRSimple 1


100%|██████████| 480/480 [00:00<00:00, 7660.28it/s]

world_BmwX5Simple 1



100%|██████████| 480/480 [00:00<00:00, 4265.23it/s]

world_CitroenCZeroSimple 1


...


100%|██████████| 480/480 [00:00<00:00, 4264.01it/s]

Only positive:
data: 9600
label: 9600



100%|██████████| 3948/3948 [00:00<00:00, 6424.88it/s]

With negative:
data: 17496
label: 17496


load_data_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/db_aug_v4_v_500_20181031.pickle
load_ndata_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/ndb_aug_v2_20181020.pickle
out_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/db_aug_v4_v_500_20181031_ndb_aug_v2_20181020_rs2
Total data: 17496
Total label: 17496
0
np_data_filtered.shape (7896, 256, 3)
data_train.shape (4737, 256, 3)
data_test.shape (1579, 256, 3)
data_vali.shape (1580, 256, 3)
1
np_data_filtered.shape (4800, 256, 3)
data_train.shape (2880, 256, 3)
data_test.shape (960, 256, 3)
data_vali.shape (960, 256, 3)
2
np_data_filtered.shape (1920, 256, 3)
data_train.shape (1152, 256, 3)
data_test.shape (384, 256, 3)
data_vali.shape (384, 256, 3)
3
np_data_filtered.shape (960, 256, 3)
data_train.shape (576, 256, 3)
data_test.shape (192, 256, 3)
data_vali.shape (192, 256, 3)
4
np_data_filtered.shape (1920, 256, 3)
data_train.shape (1152, 256, 3)
data_tes